In [ ]:
# find non-functioning threads
import pandas as pd
import time
from IPython.display import clear_output
import random
import io
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#The performance class tracks the performance of scraping and rests threads when neccesary.}

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver.v2 as uc
import random

from bs4 import BeautifulSoup
import json
import os
import re

#convert url
def elimP(string):
    rem=string[string.find("&cstart"):string.find("e=80")+4]
    return(string.replace(rem,""))
def ReFind(string, char):
    for i in range(0,len(string)):
        if(string[len(string)-i-1]==char):
            return(len(string)-i-1)
    return(-1)
def convertURL(url):
    ext=url[url.find("&cstart")-4:url.find("&cstart")]
    a=url[: ReFind(url,":")] + ext + url[ReFind(url,":"):]
    return(elimP(a))

#extract chart from html:



def getDate(HTML):
    try:
        return int(HTML[HTML.find('class="gsc_oci_g_a"')-6:HTML.find('class="gsc_oci_g_a"')-2])
    except ValueError:
        return None

def getCites(HTML):
    red = (HTML[HTML.find('class="gsc_oci_g_al"'):HTML.find('class="gsc_oci_g_al"')+30])
    cite = red[red.find(">")+1:red.find("<")]
    return int(cite) if cite.isdigit() else 0

def UpdateHTML(HTML):
    HTML = HTML[HTML.find('class="gsc_oci_g_al"')+20:]
    return HTML

def CheckMore(HTML):
    return HTML.find('class="gsc_oci_g_a"') != -1

def getDat(HTML):
    year = getDate(HTML)
    cites = getCites(HTML)
    if year:  # Only return data if a year is found
        return {"year": year, "cites": cites}
    return None


def getChart(HTML, author, url):
    years = []
    cites = []
    
    more = CheckMore(HTML)
    while more:
        data = getDat(HTML)
        if data:
            if data["year"] not in years:
                years.append(data["year"])
                cites.append(data["cites"])
            else:
                index = years.index(data["year"])
                cites[index] += data["cites"]
        
        HTML = UpdateHTML(HTML)
        more = CheckMore(HTML)
    
    # Construct the DataFrame
    Chart = pd.DataFrame({
        'Author_Name': [author] * len(years),
        'url': [url] * len(years),
        'year': years,
        'cites': cites
    })
    
    return Chart
def extract_relevant_html(html_content):
    """
    Extract only the relevant parts of the HTML content.
    """
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    # Return the cleaned HTML
    return str(soup)


#check if functioning:

def save_html_to_file(content, author, url, filename):
    """
    Save the provided content to a file with metadata embedded in the delimiters.
    """
    start_delimiter = f"\n<BEGINOFPAGE: Author={author}, URL={url}>\n"
    end_delimiter = f"\n<ENDOFPAGE: Author={author}, URL={url}>\n"

    # Ensure the directory exists
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

    try:
        with open(filename, 'a', encoding='utf-8') as file:
            file.write(start_delimiter + content + end_delimiter)
        print(f"Content saved to {filename}")
    except Exception as e:
        print(f"Error {e} writing to {filename}.")
        with open(filename, 'w', encoding='utf-8') as file:
             file.write(start_delimiter + content + end_delimiter)
        print(f"Created an empty file named {filename}")




from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

from bs4 import BeautifulSoup

from bs4 import BeautifulSoup

def check_standard_from_html(html_content, expected_html_length=74000, threshold=0.15):
    """
    Check if the given HTML content meets the standard based on the presence of certain elements.
    
    Parameters:
    - html_content: HTML content as a string.
    - expected_html_length: Expected length of the HTML content.
    - threshold: Percentage of checks that need to pass for the page to be considered functioning.
    
    Returns:
    - True if the page meets the standard, False otherwise.
    - A matrix describing which parameters passed or failed.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    
    checks = {
        "[text*='[PDF] from']": False,
        "[text*='Authors']": False,
        "[text*='Publication date']": False,
        "[text*='Journal'], [text*='Conference']": False,
        "[text*='Description']": False,
        "[text*='Scholar articles']": False,
        "[text*='Volume']": False,
        "[text*='Issue']": False,
        "[text*='Pages']": False,
        "[text*='Publisher']": False,
        "[text*='Total citations']": False,
        "[text*='Cited by']": False,
        "HTML Length": False
    }
    
    checks["[text*='[PDF] from']"] = bool(soup.find(string=lambda text: '[PDF] from' in text))
    checks["[text*='Authors']"] = bool(soup.find(string=lambda text: 'Authors' in text))
    checks["[text*='Publication date']"] = bool(soup.find(string=lambda text: 'Publication date' in text))
    checks["[text*='Journal'], [text*='Conference']"] = bool(soup.find(string=lambda text: 'Journal' in text or 'Conference' in text))
    checks["[text*='Description']"] = bool(soup.find(string=lambda text: 'Description' in text))
    checks["[text*='Scholar articles']"] = bool(soup.find(string=lambda text: 'Scholar articles' in text))
    checks["[text*='Volume']"] = bool(soup.find(string=lambda text: 'Volume' in text))
    checks["[text*='Issue']"] = bool(soup.find(string=lambda text: 'Issue' in text))
    checks["[text*='Pages']"] = bool(soup.find(string=lambda text: 'Pages' in text))
    checks["[text*='Publisher']"] = bool(soup.find(string=lambda text: 'Publisher' in text))
    checks["[text*='Total citations']"] = bool(soup.find(string=lambda text: 'Total citations' in text))
    checks["[text*='Cited by']"] = bool(soup.find(string=lambda text: 'Cited by' in text))
    
    # Check for length of HTML content
    lower_bound = 0.9 * expected_html_length
    upper_bound = 1.1 * expected_html_length
    checks["HTML Length"] = lower_bound <= len(html_content) <= upper_bound
    
    passed_checks = sum(1 for check, passed in checks.items() if passed)
    total_checks = len(checks)
    passed_percentage = passed_checks / total_checks
    
    return passed_percentage >= threshold, checks




def check_html_pages(filename):
    """
    Check the percentage of HTML pages in the file that meet the standard.
    
    Parameters:
    - filename: Name of the file containing saved HTML content.
    
    Returns:
    - Percentage of pages that meet the standard.
    """
    # Read the file content
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # Split the content using the delimiters
    pages = content.split("<ENDOFPAGE:")
    pages = [page.split("<BEGINOFPAGE:")[1] for page in pages if "<BEGINOFPAGE:" in page]

    # Check each page
    passed_count = 0
    for page in pages:
        # Extract the HTML content from the delimiter
        html_content = page.split(">\n")[1]  # Corrected this line
        
        passed, matrix = check_standard_from_html(html_content)
        if passed:
            passed_count += 1
        
        # Print the matrix for each page
        print(f"HTML Content Length: {len(html_content)}")
        for check, result in matrix.items():
            print(f"Check: {check}, Passed: {result}")
        print("------")

    # Calculate the percentage of pages that passed
    percentage_passed = (passed_count / len(pages)) * 100
    return percentage_passed


def selenium_request(url, PROXY):
    """
    Make a request to the given URL using Selenium with a specified proxy and a random user agent.

    :param url: The URL to navigate to.
    :param PROXY: The proxy to use for the request.
    :return: The Selenium WebDriver object after navigating to the URL.
    """
    
    # Set up the proxy for Chrome
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": PROXY,
        "ftpProxy": PROXY,
        "sslProxy": PROXY,
        "proxyType": "MANUAL"
    }

    # Define user agents and select one randomly
    user_agents = [ 
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
        'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
        'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
    ]
    user_agent = random.choice(user_agents)

    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument(f'user-agent={user_agent}')

    # Initialize the Chrome driver
    driver = uc.Chrome(options=chrome_options, executable_path="chromedriver.exe")

    # Navigate to the URL
    driver.get(url)

    return driver


class Performance:
    
    
    
    def __init__(self, thread_count, avg_req_rate, max_req_rate):
        
        """
        Initialize the Performance object with the given parameters.
        Create a DataFrame to track the performance of each thread.
        Initialize variables to track overall performance.
        """
        
        # Initialize performance variables
        self.thread_count = thread_count
        self.avg_req_rate = avg_req_rate
        self.max_req_rate = max_req_rate

        # Initialize data frame to track performance of each thread
        self.thread_performance = pd.DataFrame({
            'thread_id': range(thread_count),
            'thread_requests': [0]*thread_count,
            'thread_errors': [0]*thread_count,
            'thread_success': [0]*thread_count,
            'thread_success_rate': [1]*thread_count,
            'consecutive_errors': [1]*thread_count,
            'thread_error_logs': [{}]*thread_count
        })

        # Initialize variables to track overall performance
        self.all_requests = 0
        self.all_errors = 0
        self.all_success = 0
        self.start_time = time.time()
        
    
    
    
    def add_success(self, thread_id):
        
        """
        Handle a successful request.
        Update the performance metrics for the given thread and overall.
        Adjust the average request rate based on the success rate.
        Wait between requests to maintain the average request rate.
        """
        
        # Increment the number of successful requests for the thread
        self.thread_performance.loc[thread_id, 'thread_success'] += 1
        self.thread_performance.loc[thread_id, 'thread_requests'] += 1

        # Increment the total number of successful requests
        self.all_success += 1
        self.all_requests += 1

        # Update the success rate for the thread
        self.thread_performance.loc[thread_id, 'thread_success_rate'] = self.thread_performance.loc[thread_id, 'thread_success'] / self.thread_performance.loc[thread_id, 'thread_requests']

        # Reset the number of consecutive errors for the thread
        self.thread_performance.loc[thread_id, 'consecutive_errors'] = 0
        
        # Calculate the time since the last request
        time_since_last_request = time.time() - self.start_time

        # Calculate the average time between requests
        avg_time_between_requests = time_since_last_request / self.all_requests

        # If the average time between requests is less than the desired average request rate, wait for a bit
        if avg_time_between_requests < 1/self.avg_req_rate:
            time_to_wait = 1/self.avg_req_rate - avg_time_between_requests
            time.sleep(time_to_wait)

        # If the number of requests is a multiple of a certain number (e.g. 100), take a longer break
        if self.all_requests % 100 == 0:
            time.sleep(10)

        # If the success rate is too low, decrease the average request rate
        if self.all_success / self.all_requests < 0.9:
            self.avg_req_rate *= 0.9

        # If the success rate is high and the average request rate is less than the maximum request rate, increase the average request rate
        if self.all_success / self.all_requests > 0.99 and self.avg_req_rate < self.max_req_rate:
            self.avg_req_rate *= 1.1
    
    
    
    
    
    
    
    def add_error(self, thread_id, error):
        
        """
        Handle an error.
        Update the performance metrics for the given thread and overall.
        Log the error and the time it occurred.
        Wait between requests to maintain the average request rate.
        If consecutive errors occur, increase the wait time exponentially.
        """
        
        # Increment the number of errors for the thread
        self.thread_performance.loc[thread_id, 'thread_errors'] += 1
        self.thread_performance.loc[thread_id, 'thread_requests'] += 1

        # Increment the total number of errors
        self.all_errors += 1
        self.all_requests += 1

        # Update the success rate for the thread
        self.thread_performance.loc[thread_id, 'thread_success_rate'] = self.thread_performance.loc[thread_id, 'thread_success'] / self.thread_performance.loc[thread_id, 'thread_requests']

        # Log the error
        error_time = time.time() - self.start_time
        self.thread_performance.at[thread_id, 'thread_error_logs'][error_time] = error
        
        
        # If the number of errors is too high, increase the wait time exponentially
        if self.thread_performance.loc[thread_id, 'thread_errors'] > 5:
            # Increase the number of consecutive errors
            self.thread_performance.loc[thread_id, 'consecutive_errors'] += 1
            # Calculate the wait time as 2 to the power of the number of consecutive errors
            wait_time = 2 ** self.thread_performance.loc[thread_id, 'consecutive_errors']
            time.sleep(wait_time)

        # If the success rate is too low, decrease the average request rate
        if self.all_success / self.all_requests < 0.9:
            self.avg_req_rate *= 0.9

        # If the number of errors is a multiple of a certain number (e.g. 10), take a longer break
        if self.all_errors % 10 == 0:
            time.sleep(30)
    
    
    
    
    
    
    def __repr__(self):
        
        """
        Calculate performance metrics such as the average request rate, 
        average error rate, average success rate, overall success rate, 
        and overall error rate.
        Return a string representation of these metrics and the performance 
        of each thread.
        """


        # Clear the previous output
        clear_output(wait=True)

        # Calculate the total elapsed time
        elapsed_time = time.time() - self.start_time

        # Calculate the average request rate
        avg_request_rate = self.all_requests / elapsed_time

        # Calculate the average error rate
        avg_error_rate = self.all_errors / elapsed_time

        # Calculate the average success rate
        avg_success_rate = self.all_success / elapsed_time

        # Calculate the overall success rate
        overall_success_rate = self.all_success / self.all_requests if self.all_requests > 0 else 0

        # Calculate the overall error rate
        overall_error_rate = self.all_errors / self.all_requests if self.all_requests > 0 else 0

        # Return a string representation of the performance metrics
        return f"Performance Metrics:\n" \
               f"---------------------\n" \
               f"Elapsed Time: {elapsed_time} seconds\n" \
               f"Average Request Rate: {avg_request_rate} requests/second\n" \
               f"Average Error Rate: {avg_error_rate} errors/second\n" \
               f"Average Success Rate: {avg_success_rate} successes/second\n" \
               f"Overall Success Rate: {overall_success_rate}\n" \
               f"Overall Error Rate: {overall_error_rate}\n" \
               f"Thread Performance:\n" \
               f"{self.thread_performance}"
            # Store the performance metrics in a log file
        with open('performance_log.txt', 'a') as f:
            f.write(performance_metrics + '\n')
    


"""
Initialize the ResponseContainer with the given parameters.

:param url: The URL for which the response or error is recorded.
:param author: The author associated with the URL.
:param response: The response object from the request (default is None).
:param error: The error message if an error occurred (default is None).
"""

class ResponseContainer:
    def __init__(self, url, author, response=None, error=None):
        self.url = url
        self.author = author
        self.response = response
        self.error = error

    def is_successful(self):
        return self.response is not None and self.error is None
    
    
    
import random
import unittest
import lxml
import pandas as pd
import unittest
from unittest.mock import patch, Mock



def make_request_with_proxy(url, author, thread_id, user_agents, performance, proxy_list):
    """
    Make a request to a given URL using a dedicated proxy and a random user agent.
    Return a ResponseContainer object containing the response or error.

    :param url: The URL to make the request to.
    :param author: The author associated with the URL.
    :param thread_id: The ID of the thread making the request.
    :param user_agents: A list of user agents to choose from.
    :param performance: The Performance object to track the performance metrics.
    :param proxy_list: A list of proxies to use for the requests.
    :return: A ResponseContainer object containing the response or error.
    """
    try:
        print(performance)
    except:
        pass
    proxy_url = proxy_list[thread_id % len(proxy_list)]  # Assign a proxy based on thread_id
    try:
        driver = selenium_request(url, proxy_url)
        time.sleep(0.02)
        page_title = driver.title
        page_source = driver.page_source
        relevant_content = page_source
        # If not a 404 error, attempt to parse the chart data
        # Define a filename based on the URL or any other criteria
        filename = os.path.join("saved_html", f"thread_{thread_id}.html")
        save_html_to_file(relevant_content,author,url, filename)
        driver.quit()

        # Check if the title or content indicates a 404 error first
        if "404" in page_title or "Not Found" in page_title or "404" in page_source:
            performance.add_error(thread_id, "404 Not Found")
            return ResponseContainer(url, author, error="404 Not Found")
        chart_data = getChart(relevant_content,author,url)
        # If the chart data is empty or not as expected, treat it as an error
        if not chart_data.empty:
            performance.add_success(thread_id)
            return ResponseContainer(url, author, response=page_source)
        else:
            # Check for other common error messages
            if "500 Internal Server Error" in relevant_content:
                error_message = "500 Internal Server Error"
            elif "503 Service Unavailable" in relevant_content:
                error_message = "503 Service Unavailable"
            elif "Content not available" in relevant_content:
                error_message = "Placeholder: Content not available"
            elif(check_standard_from_html(page_source)[0]):
                error_message = "webpage seems off as it did not pass tests."
                performance.add_error(thread_id, error_message)
                return ResponseContainer(url, author, error=error_message)
            else:
                # If no specific error is identified, log a generic message
                error_message = "Failed to extract chart data and couldn't identify a specific error."
                return ResponseContainer(url, author, error=error_message)
        performance.add_error(thread_id, error_message)
        return ResponseContainer(url, author, error=error_message)
    except Exception as e:
        performance.add_error(thread_id, str(e))
        return ResponseContainer(url, author, error=str(e))












class ChartDataScraper:

    def __init__(self, csv_path, proxy_list, user_agents, performance_params, batch_size):
        """
        Initialize the ChartDataScraper object with the given parameters.

        :param csv_path: Path to the CSV file containing URLs and authors.
        :param proxy_list: A list of proxies to use for the requests.
        :param user_agents: A list of user agents to choose from.
        :param performance_params: Parameters to initialize the Performance class.
        :param batch_size: The number of URLs to process in each batch.
        """
        
        # Load the CSV into a DataFrame
        self.dataframe = pd.read_csv(csv_path, low_memory=False)
        
        # Initialize other attributes
        self.proxy_list = proxy_list
        self.user_agents = user_agents
        self.batch_size = batch_size
        
        # Create an instance of the Performance class
        self.performance = Performance(1,0.1,0.2)
        
        # Initialize containers for storing results and errors
        self.master_data = pd.DataFrame(columns=['Author_Name', 'url', 'chart_data'])
        self.non_working_urls = []
        self.misc_errors = []
        self.parsing_errors = []
    def scrape_batch(self, batch_urls):
        """
        Scrape a batch of URLs and extract chart data.

        :param batch_urls: A list of URLs to scrape in the current batch.
        """
        for _, url_data in batch_urls.iterrows():
            url = url_data['url']
            author = url_data['Author_Name']
            thread_id = hash(author) % self.performance.thread_count  # Distribute URLs based on author's hash

            try:
                # Use the make_request_with_proxy function to scrape the URL
                response_container = make_request_with_proxy(url, author, thread_id, self.user_agents, self.performance, self.proxy_list)

                # If the URL returns a 404 error, add it to non_working_urls
                if response_container.error and "404 Not Found" in response_container.error:
                    self.non_working_urls.append(url)

                # If the URL returns any other error, add it to misc_errors
                elif response_container.error:
                    self.misc_errors.append((url, response_container.error))

                # If the URL is successfully scraped, extract chart data
                else:
                    try:
                        chart_data = getChart(response_container.response, author, url)

                        # Drop the 'author' and 'url' columns from the chart_data DataFrame
                        if 'author' in chart_data.columns:
                            chart_data = chart_data.drop(columns=['Author_Name'])
                        if 'url' in chart_data.columns:
                            chart_data = chart_data.drop(columns=['url'])

                        # Convert the chart_data DataFrame to a JSON string
                        chart_data_json = chart_data.to_json(orient='records')

                        new_data = pd.DataFrame({
                            'Author_Name': [author],
                            'url': [url],
                            'chart_data': [chart_data_json]
                        })
                        self.master_data = self.master_data.append(new_data, ignore_index=True)
                    except Exception as e:
                        print(str(e))
                        # If chart data extraction fails, add the URL to parsing_errors
                        self.parsing_errors.append((url, str(e)))

            except Exception as e:
                print(str(e))
                # Handle unexpected errors during the request process
                self.misc_errors.append((url, str(e)))

    
    def save_data(self, filename):
        """
        Save the master_data to a CSV file with the given filename.

        :param filename: The name of the file to save the data to.
        """
        try:
            # Assuming master_data is a DataFrame
            self.master_data.to_csv(filename, index=False)
            print(f"Data successfully saved to {filename}")
        except Exception as e:
            print(f"Error saving data to {filename}. Error: {str(e)}")
    
    def save_errors(self):
        """
        Save the lists of URLs that resulted in errors to separate files.
        """
        with open("non_working_urls.txt", "w") as f:
            for url in self.non_working_urls:
                f.write(url + "\n")

        with open("misc_errors.txt", "w") as f:
            for url, error in self.misc_errors:
                f.write(f"{url} - {error}\n")

        with open("parsing_errors.txt", "w") as f:
            for url in self.parsing_errors:
                f.write(url + "\n")
                
    def run(self):
        """
        Orchestrates the scraping process by dividing the dataframe into batches and processing each batch.
        After every few batches, the scraped data is saved to a CSV file.
        """
        # Calculate the total number of batches
        total_batches = len(self.dataframe) // self.batch_size + (len(self.dataframe) % self.batch_size != 0)

        # Iterate through each batch
        for batch_num in range(total_batches):
            start_index = batch_num * self.batch_size
            end_index = start_index + self.batch_size
            batch_urls = self.dataframe.iloc[start_index:end_index]

            # Scrape the current batch
            self.scrape_batch(batch_urls)

            # Save the scraped data after every few batches (e.g., every 10 batches)
            if (batch_num + 1) % 10 == 0:
                self.save_data(f"scraped_data_batch_{batch_num + 1}.csv")

        # Save any remaining data after all batches are processed
        self.save_data("final_scraped_data.csv")

        # Save the errors encountered during the scraping process
        self.save_errors()
        
    def resume(self, last_processed_index):
        """
        Resume the scraping process from the batch after the last_processed_index.

        :param last_processed_index: The index of the last processed URL before the interruption.
        """
        # Calculate the starting batch index based on the last_processed_index and batch_size
        start_batch_index = (last_processed_index // self.batch_size) + 1

        # Split the dataframe into batches starting from start_batch_index
        total_batches = len(self.dataframe) // self.batch_size + (1 if len(self.dataframe) % self.batch_size else 0)

        for batch_index in range(start_batch_index, total_batches):
            start_index = batch_index * self.batch_size
            end_index = (batch_index + 1) * self.batch_size
            batch_urls = self.dataframe.iloc[start_index:end_index]['url'].tolist()

            # Scrape the current batch
            self.scrape_batch(batch_urls)

            # Optionally, save the data after every few batches (for example, every 10 batches)
            if (batch_index + 1) % 10 == 0:
                self.save_data(f"data_batch_{batch_index + 1}.csv")

        # Save any remaining errors after all batches are processed
        self.save_errors()


'''
class MockOpen(unittest.mock.mock_open):
    def __init__(self, *args, **kwargs):
        self.mock_file = kwargs.pop('mock_file', Mock(spec=io.TextIOWrapper))
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        return self.mock_file
'''

import unittest
from bs4 import BeautifulSoup
from unittest.mock import patch, Mock
import io
"""
class TestScraperFunctions(unittest.TestCase):
    

    def test_getYears(self):
        sample_html = '<span class="gsc_oci_g_t" style="left:0px">2017</span>'
        self.assertEqual(getYears(BeautifulSoup(sample_html, 'html.parser')), [2017])

    def test_getCitationsForYear(self):
        sample_html = '''<a href="https://scholar.google.com/scholar?oi=bibs&amp;hl=en&amp;cites=8632027577153898602&amp;as_sdt=5&amp;as_ylo=2017&amp;as_yhi=2017" class="gsc_oci_g_a" style="left:5px;height:29px;top:28px;z-index:6"><span class="gsc_oci_g_al">1</span></a>'''
        soup = BeautifulSoup(sample_html, 'html.parser')
        self.assertEqual(getCitationsForYear(soup, 2017), 1)

    def test_getChart(self):
        sample_html = '''<div class="gsc_oci_value"><div style="margin-bottom:1em"><a href="https://scholar.google.com/scholar?oi=bibs&amp;hl=en&amp;cites=8632027577153898602&amp;as_sdt=5">Cited by 6</a></div><div id="gsc_oci_graph_wrapper"><div id="gsc_oci_graph" style="width:195px;"><div id="gsc_oci_graph_x"></div><div id="gsc_oci_graph_bars"><span class="gsc_oci_g_t" style="left:0px">2017</span><a href="https://scholar.google.com/scholar?oi=bibs&amp;hl=en&amp;cites=8632027577153898602&amp;as_sdt=5&amp;as_ylo=2017&amp;as_yhi=2017" class="gsc_oci_g_a" style="left:5px;height:29px;top:28px;z-index:6"><span class="gsc_oci_g_al">1</span></a></div></div></div>'''
        expected_output = pd.DataFrame({"year": [2017], "cites": [1]})
        pd.testing.assert_frame_equal(getChart(sample_html), expected_output)
"""


import os
import sys

class TestDataScraper(unittest.TestCase):

    def setUp(self):
        # Sample data for testing
        self.csv_path = "AuthorsPapersFixed.csv"
        self.proxy_list = ["192.187.111.82:17045"]
        self.user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
        ]
        self.performance_params = {"thread_count": 1, "avg_req_rate": 0.1, "max_req_rate": 0.2}
        self.batch_size = 5

    def test_selenium_request(self):
        # Load a sample URL from AuthorsPapers.csv
        sample_url = pd.read_csv(self.csv_path).iloc[0]['url']
        author = pd.read_csv(self.csv_path).iloc[0]['Author_Name']

        # Use the selenium_request function to navigate to the provided URL
        driver = selenium_request(sample_url, self.proxy_list[0])
        page_source = driver.page_source
        driver.quit()
        

        # Parse the page source using your HTML parsing functions
        chart_data = getChart(page_source,author,sample_url)
        print(chart_data)

        # Check if the chart data is not empty
        self.assertFalse(chart_data.empty, "Chart data is empty")

    def test_make_request_with_proxy(self):
        # Load a sample URL and corresponding author from AuthorsPapers.csv
        sample_data = pd.read_csv(self.csv_path).iloc[0]
        sample_url = sample_data['url']
        author = sample_data['Author_Name']

        # Use the make_request_with_proxy function to make a request to the provided URL
        response_container = make_request_with_proxy(sample_url, author, 0, self.user_agents, Performance(**self.performance_params), self.proxy_list)

        # If the response is successful, parse the HTML content using your HTML parsing functions
        if response_container.is_successful():
            chart_data = getChart(response_container.response,author,sample_url)

            # Check if the chart data is not empty
            self.assertFalse(chart_data.empty, "Chart data is empty")

    def test_scrape_batch(self):
        # Initialize the ChartDataScraper class with the required parameters
        scraper = ChartDataScraper(self.csv_path, self.proxy_list, self.user_agents, self.performance_params, self.batch_size)

        # Load a batch of URLs from AuthorsPapers.csv
        batch_urls = pd.read_csv(self.csv_path).iloc[:self.batch_size]

        # Use the scrape_batch method to scrape the batch
        scraper.scrape_batch(batch_urls)
        print(scraper.parsing_errors)
        print(scraper.misc_errors)

        # Check the master_data attribute of the ChartDataScraper instance for the scraped data
        self.assertFalse(scraper.master_data.empty, "Master data is empty after scraping")
def create_test_csv(original_csv, test_csv, num_rows):
    """
    Create a smaller CSV file with a specified number of rows.

    :param original_csv: Path to the original CSV file.
    :param test_csv: Path to the smaller CSV file to be created.
    :param num_rows: Number of rows to include in the smaller CSV.
    """
    df = pd.read_csv(original_csv)
    df.iloc[:num_rows].to_csv(test_csv, index=False)


def main():
# for testing    
#    original_csv="AuthorsPapersFixed.csv"
#    create_test_csv("AuthorsPapersFixed.csv","test.csv",100)
    
    # Parameters
    csv_path = "AuthorsPapersFixed.csv"
    proxy_list = ["192.187.111.82:17045"]
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    ]
    performance_params = {"thread_count": 1, "avg_req_rate": 0.135, "max_req_rate": 0.2}
    batch_size = 5

    # Initialize the ChartDataScraper
    scraper = ChartDataScraper(csv_path, proxy_list, user_agents, performance_params, batch_size)

    # Run the scraper
    scraper.run()

    # Optionally, print out any errors encountered
    print("Master data", master_data)
    print("Non-working URLs:", scraper.non_working_urls)
    print("Miscellaneous Errors:", scraper.misc_errors)
    print("Parsing Errors:", scraper.parsing_errors)





    
    
    
    
    
if __name__ == "__main__":
    main()


Performance Metrics:
---------------------
Elapsed Time: 1556914.9850552082 seconds
Average Request Rate: 7.001024529039064e-05 requests/second
Average Error Rate: 2.183805816397506e-05 errors/second
Average Success Rate: 4.817218712641558e-05 successes/second
Overall Success Rate: 0.6880733944954128
Overall Error Rate: 0.3119266055045872
Thread Performance:
   thread_id  thread_requests  thread_errors  thread_success  \
0          0              109             34              75   

   thread_success_rate  consecutive_errors  \
0             0.688073                   0   

                                   thread_error_logs  
0  {123.01619100570679: 'webpage seems off as it ...  
Content saved to saved_html\thread_0.html


C:\Users\dylan\anaconda3\lib\unittest\case.py:628: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  testMethod()


<unittest.runner.TextTestResult run=3 errors=0 failures=1>

In [13]:
pip install --upgrade pip

  Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
Note: you may need to restart the kernel to use updated packages.
